# Links Analysis

### Import

In [ ]:
from networkx import hits
import numpy as np
import networkx as nx

### Fonctions

Matrice d'adjacence

In [69]:
def create_numpy_adjacency_matrix(gml_file):
    graph = nx.read_gml(gml_file)
    adjacency_matrix = nx.to_numpy_array(graph)
    return adjacency_matrix, list(graph.nodes())

Voisins communs

In [70]:
def voisins_communs(matrix):
    # Initialiser une matrice pour stocker les voisins communs
    voisins = np.zeros(matrix.shape)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            voisins[i, j] = np.sum(np.multiply(matrix[i, :], matrix[j, :]))
    return voisins

Matrice d'attachement préférentiel

In [71]:
def preferential_attachment(matrix):
    degrees = np.sum(matrix, axis=1)
    pref_attach = np.zeros(matrix.shape) 
    # Parcourir les paires de nœuds
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            # L'attachement préférentiel entre deux nœuds est le produit de leurs degrés
            pref_attach[i, j] = degrees[i] * degrees[j]
    return pref_attach 

Matrice de similarité par cosinus

In [72]:
def cosine_similarity(matrix):
    cosine_sim = np.zeros(matrix.shape)
    # Parcourir les paires de nœuds
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            # Calculer le produit scalaire des deux vecteurs correspondant aux nœuds i et j
            dot_product = np.dot(matrix[i, :], matrix[j, :])
            # Calculer les normes des vecteurs pour normalisation
            norm_i = np.linalg.norm(matrix[i, :])
            norm_j = np.linalg.norm(matrix[j, :])         
            # Calculer la similarité cosinus (éviter les divisions par zéro)
            if norm_i > 0 and norm_j > 0:
                cosine_sim[i, j] = dot_product / (norm_i * norm_j)
            else:
                cosine_sim[i, j] = 0  # Si un vecteur a une norme nulle, la similarité est 0
    return cosine_sim

Matrice de similarité par Jaccard

In [73]:
def jaccard_similarity(matrix):
    n = matrix.shape[0]  # Nombre de nœuds
    degrees = np.sum(matrix, axis=1)  # Degré de chaque nœud
    sim_common = np.dot(matrix, matrix.T)  # Nombre de voisins communs entre chaque paire de nœuds
    sim_jac = np.zeros((n, n))  
    # Parcourir chaque paire de nœuds
    for i in range(n):
        for k in range(n):
            # Calculer la mesure de Jaccard (éviter la division par zéro)
            denominator = degrees[i] + degrees[k] - sim_common[i, k]
            if denominator > 0:
                sim_jac[i, k] = sim_common[i, k] / denominator
            else:
                sim_jac[i, k] = 0   
    return sim_jac

Matrice de similarité par Dice

In [74]:
def dice_similarity(matrix):
    n = matrix.shape[0]  # Nombre de nœuds
    degrees = np.sum(matrix, axis=1)  # Degré de chaque nœud
    sim_common = np.dot(matrix, matrix.T)  # Nombre de voisins communs entre chaque paire de nœuds
    sim_dice = np.zeros((n, n))
    
    # Parcourir chaque paire de nœuds
    for i in range(n):
        for k in range(n):
            # Calculer la mesure de Dice (éviter la division par zéro)
            denominator = degrees[i] + degrees[k]
            if denominator > 0:
                sim_dice[i, k] = 2 * sim_common[i, k] / denominator
            else:
                sim_dice[i, k] = 0  
    return sim_dice

Matrice de Katz

In [86]:
def Katz(graph, alpha):
    #Calculer la matrice de Katz
    A = graph
    n = A.shape[0] # nombre de noeuds
    I = np.identity(n) # matrice identité
    Katz = np.linalg.inv(I - alpha*A) - I # matrice de Katz, on calcule d'abord l'inverse de I - alpha*A puis on soustrait I
    print(f"Ceci est la matrice de Katz :\n {Katz}\n")
    return Katz

Probabilité de transition

In [76]:
def transition_proba(graph):
    A = graph
    n = A.shape[0] # nombre de noeuds
    degrees = [sum(A[i]) for i in range(n)] # degré de chaque noeud
    transition_graph = np.zeros((n,n))
    """
    au début je n'avais pas mis (transition_graph) mais c'est nécessaire sinon on modifie la matrice A pendant
    le calcul et on obtient un matrice null
    """

    #Calculer la matrice de transition
    for i in range(len(A)):
        for k in range(len(A)):
            transition_graph[i,k] = A[i,k]/degrees[i]
    return transition_graph

Matrice FPT et CT

In [92]:
def matrice_FPT_et_CT(graph):
	# Calculer la pseudo-inverse de la matrice Laplacienne
	A = graph
	n = A.shape[0]  # nombre de noeuds
	degrees = [sum(A[i]) for i in range(n)]  # degré de chaque noeud
	D = np.diag(degrees)  # matrice diagonale des degrés
	L = D - A  # matrice Laplacienne
	eeT = np.ones((n, n))  # matrice de 1
	Lplus = np.linalg.inv((L - eeT / n)) + (eeT / n)

	# Calculer la matrice de FPT
	FPT = np.zeros((n, n))
	for i in range(len(A)):
		for k in range(len(A)):
			if i != k:
				sum_term = 0
				for j in range(len(A)):
					sum_term += (Lplus[i, j] - Lplus[i, k] - Lplus[k, j] + Lplus[k, k]) * D[j, j]
				FPT[k, i] = sum_term
	FPT_transpose = np.transpose(FPT)

	CT = FPT_transpose + FPT

	return FPT_transpose, CT

Score Hub

In [102]:
def scores_Hub_Authority(graph):
    # Calculer les scores HITS
    hub_scores, authority_scores = nx.hits(graph)
    
    # Afficher les résultats
    print(f"Ceci est le score de Hub :\n {hub_scores}\n")
    return hub_scores, authority_scores

### Applications

In [97]:
# Chemin vers le fichier GML
gml_file = 'graph.gml'
# Créer la matrice d'adjacence
adjacency_matrix, nodes = create_numpy_adjacency_matrix(gml_file)

# Enregistrer la matrice d'adjacence dans un fichier
np.savetxt('outputmatrix/adjacency_matrix.txt', adjacency_matrix)

In [ ]:
cosine_sim = cosine_similarity(adjacency_matrix)
print(cosine_sim)

# Enregistrer la matrice de similarité cosinus dans un fichier
np.savetxt('outputmatrix/cosine_similarity_matrix.txt', cosine_sim)

In [ ]:
dice_sim = dice_similarity(adjacency_matrix)
print(dice_sim)

# Enregistrer la matrice de similarité Dice dans un fichier
np.savetxt('outputmatrix/dice_similarity_matrix.txt', dice_sim)

In [ ]:
jaccard_sim = jaccard_similarity(adjacency_matrix)
print(jaccard_sim)

# Enregistrer la matrice de similarité Jaccard dans un fichier
np.savetxt('outputmatrix/jaccard_similarity_matrix.txt', jaccard_sim)

In [ ]:
# Déterminer la valeur de alpha
alpha = 0.1

# Calculer la matrice de Katz
katz_matrix = Katz(adjacency_matrix, alpha)

# Enregistrer la matrice de Katz dans un fichier
np.savetxt('outputmatrix/katz_matrix.txt', katz_matrix)

In [ ]:
pref_attach = preferential_attachment(adjacency_matrix)
print(pref_attach)

# Enregistrer la matrice d'attachement préférentiel dans un fichier
np.savetxt('outputmatrix/preferential_attachment_matrix.txt', pref_attach)

In [ ]:
transition_graph = transition_proba(adjacency_matrix)
print(transition_graph)

# Enregistrer la matrice de probabilité de transition dans un fichier
np.savetxt('outputmatrix/transition_probability_matrix.txt', transition_graph)

In [93]:
# Calculer et enregistrer les matrices FPT et CT
FPT, CT = matrice_FPT_et_CT(transition_graph)
np.savetxt('outputmatrix/FPT_matrix.txt', FPT)
np.savetxt('outputmatrix/CT_matrix.txt', CT)

In [ ]:
# Calculer les scores de Hub et Authority

graph = nx.read_gml('graph.gml')

# Calcul des scores
hub_scores, authority_scores = scores_Hub_Authority(graph)

# Enregistrement des scores
np.savetxt('outputmatrix/hub_scores.txt', list(hub_scores.values()))